## Filtro FIR con respuesta en frecuencia arbitraria

Consideremos un filtro FIR 
- De orden $L$ (par) con $L+1$ coeficientes 
- Centrado en el origen (cero fase)
- Con respuesta al impulso simétrica: $h_{n} = h_{-n}$

Podemos escribir su respuesta en frecuencia como 

$$
H[k] = \sum_{n=-L/2}^{L/2} h[n] e^{-j \frac{2\pi}{N} k n} = h[0] + \sum_{n=1}^{L/2} h[n]\cos \left(\frac{2\pi}{N} k n\right)
$$
que para $k=0, \ldots, N-1$ en forma matricial es

$$ 
\begin{align} 
\begin{pmatrix} H[0] \\ H[1] \\ H[2] \\ \vdots \\ H[N-1] \\ \end{pmatrix} 
&= 
\begin{pmatrix} 
1 & 1 & 1 & \cdots & 1 \\ 
1 & \cos \left(\frac{2\pi}{N} \right) & \cos \left(\frac{2\pi}{N} 2\right) & \cdots & \cos \left(\frac{2\pi}{N} L/2\right) \\ 
1 & \cos \left(\frac{2\pi}{N} 2 \right) & \cos \left(\frac{2\pi}{N} 4 \right) & \cdots & \cos \left(\frac{2\pi}{N} 2 L/2\right) \\ 
\vdots & \dots & \dots & \ddots & \vdots \\ 
1 & \cos \left(\frac{2\pi}{N} (N-1)\right) & \cos \left(\frac{2\pi}{N} (N-1) 2\right) & \cdots & \cos \left(\frac{2\pi}{N} (N-1) L/2\right) \\ 
\end{pmatrix} 
\begin{pmatrix} h[0] \\ h[1] \\ h[2] \\ \vdots \\ h[L/2] \\ \end{pmatrix} \nonumber \\
H &= A h, \nonumber
\end{align} 
$$

donde $A \in \mathbb{R}^{NxL/2}$

Si especificamos $H$ podemos despejar $h$ pero no podemos invertir A ya que no es cuadrada.
***
Una alternativa es buscar $\hat h \approx h$ que minimice el error cuadrático medio

$$
\min_h \| H - Ah\|^2,
$$
derivando e igualando a cero obtenemos el sistema de **ecuaciones normales**

$$
\begin{align}
\frac{d}{dh} \| H - Ah\|^2 &= 0 \nonumber \\
- 2 A^T (H - Ah) &= 0 \nonumber \\
A^T A h &= A^T H \nonumber \\
\hat h &= (A^T A)^{-1} A^T H  \nonumber \\
\hat h &= A^{\dagger} H, \nonumber
\end{align}
$$

donde 
- $A^{\dagger} = (A^T A)^{-1} A^T$ se conoce como la pseudo-inversa de A
- $\hat h$ se conoce como el **estimador de mínimos cuadrados** de h

***

`scipy.signal.firls(numtaps, bands, desired, weight=None, nyq=None, fs=None)`

- numtaps (entero impar): número de coeficientes
- bands: secuencia creciente de frecuencias
- desired: secuencia creciente de ganancias
- fs (flotante): Frecuencia de muestreo

In [ ]:
# Estimador de minimos cuadrados para filtro FIR usando scipy.signal
h = scipy.signal.firls(numtaps=1000+1, 
                       bands=(0., 640, 650, 2340, 2350, 2750, 2760, sample_rate//2), 
                       desired=(5., 5., 0., 0., 1., 1., 0., 0.), fs=sample_rate)

plt.close('all'); fig = plt.figure(figsize=(8, 6))
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);
Audio(data_filt, rate=int(sample_rate))